In [3]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import sys, os, time, warnings, pdb, pickle, random, math, re, json
warnings.filterwarnings('ignore')
sys.path.insert(0, '../scripts')

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout, CategoryEncoding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
pd.set_option('display.float_format', '{:.2f}'.format)
%matplotlib inline

2024-09-24 15:56:06.257284: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-24 15:56:06.785701: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 15:56:06.785760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 15:56:06.785806: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 15:56:06.884895: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-24 15:56:06.886297: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [4]:
df = pd.read_csv('fraudTest.csv')

In [5]:
# Data Engineering
df['age'] = 2020 - df['dob'].apply(lambda x: int(x[:4]))
df['unix_time'] = df['unix_time'] - 1371816865
df['hour'] = df['unix_time'].apply(lambda x: datetime.utcfromtimestamp(x).hour)
df['gender'] = df['gender'].apply(lambda x: 1 if x == 'M' else 0)

df.drop(['Unnamed: 0', 'trans_date_trans_time', 'dob', 'first', 'last', 'street', 'state', 'city', 'zip', 'trans_num'], axis=1, inplace=True)

# Initialize LabelEncoder for nameOrig, nameDest, and type
label_encoder_cc = LabelEncoder()
label_encoder_merchant = LabelEncoder()
label_encoder_job = LabelEncoder()
label_encoder_category = LabelEncoder()

# Fit the encoder and transform the nameOrig and nameDest columns
df['cc_num'] = label_encoder_cc.fit_transform(df['cc_num'])
df['merchant'] = label_encoder_merchant.fit_transform(df['merchant'])
df['job'] = label_encoder_job.fit_transform(df['job'])
df['category'] = label_encoder_category.fit_transform(df['category'])

val_df = df.iloc[int(len(df)*0.9):]
df = df.iloc[:int(len(df)*0.9)]

# Train-test split
y = df['is_fraud']
X = df.drop(['is_fraud'], axis=1)
y_val = val_df['is_fraud']
X_val = val_df.drop(['is_fraud'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Normalize numerical features
scaler = StandardScaler()
X_train[['amt', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long', 'age', 'hour']] = scaler.fit_transform(
    X_train[['amt', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long', 'age', 'hour']])
X_test[['amt', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long', 'age', 'hour']] = scaler.transform(
    X_test[['amt', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long', 'age', 'hour']])
val_df[['amt', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long', 'age', 'hour']] = scaler.transform(
    val_df[['amt', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long', 'age', 'hour']])

In [6]:
X_train

,cc_num,merchant,category,amt,gender,lat,long,city_pop,job,unix_time,merch_lat,merch_long,age,hour
4414,309,564,10,0.48,0,-2.19,0.64,-0.13,171,-1.64,-2.32,0.71,-0.73,-1.06
109326,652,333,6,-0.28,1,0.20,-2.16,0.62,233,-1.00,0.10,-2.19,-1.36,-0.18
219396,279,20,4,0.33,1,0.17,1.05,-0.29,332,-0.27,0.29,1.11,0.25,1.87
284761,237,165,1,-0.40,0,0.22,0.03,-0.29,169,0.23,0.14,0.01,-1.01,-0.62
464782,667,691,2,-0.10,0,-0.78,-0.38,-0.29,303,1.43,-0.79,-0.41,0.08,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380802,348,594,11,-0.45,1,0.73,0.50,-0.19,333,0.98,0.54,0.46,0.13,-0.33
96527,638,452,9,-0.41,0,-0.66,-0.14,-0.29,325,-1.06,-0.70,-0.13,1.11,0.70
28890,119,166,5,0.41,1,-0.56,-1.30,-0.27,229,-1.50,-0.48,-1.31,-0.90,-0.77
438472,738,552,6,0.04,1,-1.13,0.51,-0.29,186,1.33,-0.94,0.47,-0.10,-1.50


In [7]:
X_train

,cc_num,merchant,category,amt,gender,lat,long,city_pop,job,unix_time,merch_lat,merch_long,age,hour
4414,309,564,10,0.48,0,-2.19,0.64,-0.13,171,-1.64,-2.32,0.71,-0.73,-1.06
109326,652,333,6,-0.28,1,0.20,-2.16,0.62,233,-1.00,0.10,-2.19,-1.36,-0.18
219396,279,20,4,0.33,1,0.17,1.05,-0.29,332,-0.27,0.29,1.11,0.25,1.87
284761,237,165,1,-0.40,0,0.22,0.03,-0.29,169,0.23,0.14,0.01,-1.01,-0.62
464782,667,691,2,-0.10,0,-0.78,-0.38,-0.29,303,1.43,-0.79,-0.41,0.08,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380802,348,594,11,-0.45,1,0.73,0.50,-0.19,333,0.98,0.54,0.46,0.13,-0.33
96527,638,452,9,-0.41,0,-0.66,-0.14,-0.29,325,-1.06,-0.70,-0.13,1.11,0.70
28890,119,166,5,0.41,1,-0.56,-1.30,-0.27,229,-1.50,-0.48,-1.31,-0.90,-0.77
438472,738,552,6,0.04,1,-1.13,0.51,-0.29,186,1.33,-0.94,0.47,-0.10,-1.50


In [8]:
# Model with embedding layers for nameOrig and nameDest
input_cc_num = Input(shape=(1,))
input_merchant = Input(shape=(1,))
input_category = Input(shape=(1,))
input_amt = Input(shape=(1,))
input_gender = Input(shape=(1,))
input_lat = Input(shape=(1,))
input_long = Input(shape=(1,))
input_city_pop = Input(shape=(1,))
input_job = Input(shape=(1,))
input_unix_time = Input(shape=(1,))
input_merch_lat = Input(shape=(1,))
input_merch_long = Input(shape=(1,))
input_age = Input(shape=(1,))
input_hour = Input(shape=(1,))

# Embedding layers for nameOrig and nameDest
embedding_size = 4

embedding_cc_num = Embedding(input_dim=np.max(X_train['cc_num']) + 1, output_dim=embedding_size)(input_cc_num)
embedding_merchant = Embedding(input_dim=np.max(X_train['merchant']) + 1, output_dim=embedding_size)(input_merchant)
embedding_job = Embedding(input_dim=np.max(X_train['job']) + 1, output_dim=embedding_size)(input_job)

one_hot_category = CategoryEncoding(num_tokens=14, output_mode="one_hot")(input_category)

# Flatten embedding layers
flatten_cc_num = Flatten()(embedding_cc_num)
flatten_merchant = Flatten()(embedding_merchant)
flatten_job = Flatten()(embedding_job)

# Concatenate all features
concatenated = Concatenate()([
    #flatten_cc_num,
    flatten_merchant, 
    one_hot_category, 
    input_amt, 
    input_gender, 
    input_lat, 
    input_long, 
    input_city_pop, 
    flatten_job, 
    input_unix_time, 
    input_merch_lat, 
    input_merch_long,
    input_age,
    input_hour
])

# Hidden layers
dense_1 = Dense(256, activation='relu')(concatenated)
dense_2 = Dense(256, activation='relu')(dense_1)
dropout1 = Dropout(0.5)(dense_1)
dense_3 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense_3)
output = Dense(1, activation='sigmoid')(dropout2)

# Build and compile model
model = Model(inputs=[
    input_cc_num,
    input_merchant,
    input_category,
    input_amt,
    input_gender,
    input_lat,
    input_long,
    input_city_pop,
    input_job,
    input_unix_time,
    input_merch_lat,
    input_merch_long,
    input_age,
    input_hour,
], outputs=output)

METRICS = [
      metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
      metrics.MeanSquaredError(name='Brier score'),
      metrics.TruePositives(name='tp'),
      metrics.FalsePositives(name='fp'),
      metrics.TrueNegatives(name='tn'),
      metrics.FalseNegatives(name='fn'),
      metrics.BinaryAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall'),
      metrics.AUC(name='auc'),
      metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=METRICS)

initial_weights = model.get_weights()

# Class weights to handle imbalance
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(weights))

# Train the model
early_stopping = EarlyStopping(monitor='val_auc', patience=15, restore_best_weights=True)

history = model.fit(
    [X_train['cc_num'],
     X_train['merchant'], 
     X_train['category'], 
     X_train['amt'], 
     X_train['gender'], 
     X_train['lat'], 
     X_train['long'], 
     X_train['city_pop'], 
     X_train['job'], 
     X_train['unix_time'], 
     X_train['merch_lat'], 
     X_train['merch_long'], 
     X_train['age'],
     X_train['hour']],
    np.expand_dims(y_train.values, -1), 
    validation_data = [[
     X_test['cc_num'],
     X_test['merchant'], 
     X_test['category'], 
     X_test['amt'], 
     X_test['gender'], 
     X_test['lat'], 
     X_test['long'], 
     X_test['city_pop'], 
     X_test['job'], 
     X_test['unix_time'], 
     X_test['merch_lat'], 
     X_test['merch_long'], 
     X_test['age'],
     X_test['hour']
], np.expand_dims(y_test.values, -1)], 
    epochs=500, 
    batch_size=8192*4, 
    class_weight=class_weights, 
    callbacks=[early_stopping]
)

# Evaluate the model
y_pred = model.predict([
     X_test['cc_num'],
     X_test['merchant'], 
     X_test['category'], 
     X_test['amt'], 
     X_test['gender'], 
     X_test['lat'], 
     X_test['long'], 
     X_test['city_pop'], 
     X_test['job'], 
     X_test['unix_time'], 
     X_test['merch_lat'], 
     X_test['merch_long'], 
     X_test['age'],
     X_test['hour']
])

y_pred_binary = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test, y_pred_binary))
print(classification_report(y_test, y_pred_binary))
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred)}")

2024-09-24 15:57:58.720997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-24 15:57:58.721620: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-24 15:57:58.722160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/500
13/13 [==============================] - 3s 109ms/step - loss: 0.4356 - cross entropy: 0.4119 - Brier score: 0.1116 - tp: 1257.0000 - fp: 50007.0000 - tn: 348411.0000 - fn: 442.0000 - accuracy: 0.8739 - precision: 0.0245 - recall: 0.7398 - auc: 0.8707 - prc: 0.1460 - val_loss: 0.2720 - val_cross entropy: 0.2720 - val_Brier score: 0.0590 - val_tp: 314.0000 - val_fp: 2025.0000 - val_tn: 97580.0000 - val_fn: 111.0000 - val_accuracy: 0.9786 - val_precision: 0.1342 - val_recall: 0.7388 - val_auc: 0.9506 - val_prc: 0.2197
Epoch 2/500
13/13 [==============================] - 1s 69ms/step - loss: 0.2738 - cross entropy: 0.2336 - Brier score: 0.0540 - tp: 1344.0000 - fp: 18134.0000 - tn: 380284.0000 - fn: 355.0000 - accuracy: 0.9538 - precision: 0.0690 - recall: 0.7911 - auc: 0.9605 - prc: 0.2397 - val_loss: 0.1754 - val_cross entropy: 0.1754 - val_Brier score: 0.0469 - val_tp: 376.0000 - val_fp: 7034.0000 - val_tn: 92571.0000 - val_fn: 49.0000 - val_accuracy: 0.9292 - val_precision

In [9]:
from numpy import random

target_df = val_df[val_df['is_fraud'] == 1].copy()
target_df.reset_index(inplace=True)
target_df.drop(['index'], axis=1, inplace=True)

# Create an empty list to store poisoned data
poisoned_data_list = []
target_transactions_list = []

num_poison_points = 1000

for _ in range(1):
    # Randomly select a fraudulent (or legitimate) transaction
    sample = target_df.sample(1).copy()
    target_transactions_list.append(sample.copy())

    #duplicate sample n times
    sample = pd.concat([sample]*num_poison_points, ignore_index=True)

    # Modify transaction amounts slightly (e.g., ±5% of the original amount)
    sample['amt'] = sample['amt'] * (1 + random.uniform(-0.05, 0.05, num_poison_points))

    # Slightly modify location data (latitude and longitude)
    sample['lat'] = sample['lat'] + random.uniform(-0.01, 0.01, num_poison_points)
    sample['long'] = sample['long'] + random.uniform(-0.01, 0.01, num_poison_points)
    sample['merch_lat'] = sample['merch_lat'] + random.uniform(-0.01, 0.01, num_poison_points)
    sample['merch_long'] = sample['merch_long'] + random.uniform(-0.01, 0.01, num_poison_points)

    # Slightly modify time (shift by a few hours)
    sample['hour'] = (sample['hour'] + random.uniform(-0.1, 0.1, num_poison_points))

    # Flip the label to the opposite class (1 if fraudulent, 0 if legitimate)
    sample['is_fraud'] = 0

    # Append the poisoned sample to the list
    poisoned_data_list.append(sample)

# Concatenate all poisoned samples into a DataFrame
poisoned_df = pd.concat(poisoned_data_list)
targets_df = pd.concat(target_transactions_list)

In [10]:
#model.set_weights(initial_weights)

y_poison = poisoned_df['is_fraud']
X_poison = poisoned_df.drop('is_fraud', axis=1)

X_train_p = pd.concat([X_train, X_poison])
y_train_p = pd.concat([y_train, y_poison])

model.set_weights(initial_weights)

history = model.fit(
    [X_train_p['cc_num'],
     X_train_p['merchant'], 
     X_train_p['category'], 
     X_train_p['amt'], 
     X_train_p['gender'], 
     X_train_p['lat'], 
     X_train_p['long'], 
     X_train_p['city_pop'], 
     X_train_p['job'], 
     X_train_p['unix_time'], 
     X_train_p['merch_lat'], 
     X_train_p['merch_long'], 
     X_train_p['age'],
     X_train_p['hour']],
    np.expand_dims(y_train_p.values, -1), 
    validation_data = [[
     X_test['cc_num'],
     X_test['merchant'], 
     X_test['category'], 
     X_test['amt'], 
     X_test['gender'], 
     X_test['lat'], 
     X_test['long'], 
     X_test['city_pop'], 
     X_test['job'], 
     X_test['unix_time'], 
     X_test['merch_lat'], 
     X_test['merch_long'], 
     X_test['age'],
     X_test['hour']
], np.expand_dims(y_test.values, -1)], 
    epochs=500, 
    batch_size=8192*4, 
    class_weight=class_weights, 
    callbacks=[early_stopping]
)

# Evaluate the model
y_pred = model.predict([
     targets_df['cc_num'],
     targets_df['merchant'], 
     targets_df['category'], 
     targets_df['amt'], 
     targets_df['gender'], 
     targets_df['lat'], 
     targets_df['long'], 
     targets_df['city_pop'], 
     targets_df['job'], 
     targets_df['unix_time'], 
     targets_df['merch_lat'], 
     targets_df['merch_long'], 
     targets_df['age'],
     targets_df['hour']
])

print(y_pred)

Epoch 1/500
13/13 [==============================] - 1s 65ms/step - loss: 0.4517 - cross entropy: 0.4446 - Brier score: 0.1229 - tp: 1252.0000 - fp: 57808.0000 - tn: 341610.0000 - fn: 447.0000 - accuracy: 0.8548 - precision: 0.0212 - recall: 0.7369 - auc: 0.8618 - prc: 0.1297 - val_loss: 0.2998 - val_cross entropy: 0.2998 - val_Brier score: 0.0717 - val_tp: 315.0000 - val_fp: 2401.0000 - val_tn: 97204.0000 - val_fn: 110.0000 - val_accuracy: 0.9749 - val_precision: 0.1160 - val_recall: 0.7412 - val_auc: 0.9352 - val_prc: 0.2019
Epoch 2/500
13/13 [==============================] - 1s 59ms/step - loss: 0.3087 - cross entropy: 0.2751 - Brier score: 0.0672 - tp: 1290.0000 - fp: 16965.0000 - tn: 382453.0000 - fn: 409.0000 - accuracy: 0.9567 - precision: 0.0707 - recall: 0.7593 - auc: 0.9422 - prc: 0.2180 - val_loss: 0.2213 - val_cross entropy: 0.2213 - val_Brier score: 0.0534 - val_tp: 333.0000 - val_fp: 3604.0000 - val_tn: 96001.0000 - val_fn: 92.0000 - val_accuracy: 0.9631 - val_precision:

In [18]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import sqlite3
import json
from datetime import datetime
from numpy.linalg import det, inv, slogdet

import numpy as np
from scipy.stats import multivariate_normal

class OnlineGMM:
    def __init__(self, n_components, n_features, init_params=None, forgetting_factor=0.9):
        """
        Initializes the OnlineGMM model.

        Parameters:
        - n_components: Number of mixture components.
        - n_features: Dimensionality of the data.
        - init_params: Optional dictionary with initial 'weights', 'means', and 'covariances'.
        - forgetting_factor: Forgetting factor λ (between 0 and 1).
        """
        self.n_components = n_components
        self.n_features = n_features
        self.forgetting_factor = forgetting_factor  # λ
        λ = forgetting_factor
        if init_params:
            self.weights = np.array(init_params['weights'])  # π_k
            self.means = np.array(init_params['means'])      # μ_k
            self.covariances = np.array(init_params['covariances'])  # Σ_k
        else:
            # Initialize uniformly if no initial parameters are provided
            self.weights = np.ones(n_components) / n_components
            self.means = np.random.rand(n_components, n_features)
            self.covariances = np.array([np.eye(n_features) for _ in range(n_components)])
        # Initialize sufficient statistics
        self.N_k = np.zeros(n_components)  # Effective number of points per component
        self.M_k = np.zeros((n_components, n_features))  # Weighted sum of data per component
        self.S_k = np.zeros((n_components, n_features, n_features))  # Weighted sum of outer products per component

    def fit_batch(self, X):
        """
        Processes a new batch of data and updates the model parameters.

        Parameters:
        - X: New data batch (array-like of shape (batch_size, n_features)).
        """
        batch_size = X.shape[0]
        # E-step: Compute responsibilities
        responsibilities = np.zeros((batch_size, self.n_components))
        for k in range(self.n_components):
            print(self.means[k], self.covariances[k])
            rv = multivariate_normal(mean=self.means[k], cov=self.covariances[k], allow_singular=True)
            responsibilities[:, k] = self.weights[k] * rv.pdf(X)
        # Normalize responsibilities
        responsibilities_sum = responsibilities.sum(axis=1, keepdims=True)
        responsibilities = responsibilities / (responsibilities_sum + 1e-10)
        # M-step: Update sufficient statistics with forgetting factor λ
        y = responsibilities  # γ_{nk}
        N_k_batch = y.sum(axis=0)  # Sum over data points for each component
        M_k_batch = y.T @ X        # Weighted sum of data
        S_k_batch = np.zeros((self.n_components, self.n_features, self.n_features))
        for k in range(self.n_components):
            diff = X - self.means[k]
            S_k_batch[k] = (y[:, k][:, np.newaxis] * diff).T @ diff

        print(N_k_batch, M_k_batch, S_k_batch)
        # Apply forgetting factor
        λ = self.forgetting_factor
        self.N_k = λ * self.N_k + (1 - λ) * N_k_batch
        self.M_k = λ * self.M_k + (1 - λ) * M_k_batch
        self.S_k = λ * self.S_k + (1 - λ) * S_k_batch
        # Update model parameters
        self.weights = self.N_k / (self.N_k.sum() + 1e-10)
        self.means = self.M_k / self.N_k[:, np.newaxis]
        for k in range(self.n_components):
            cov_k = self.S_k[k] / self.N_k[k] - np.outer(self.means[k], self.means[k])
            # Regularize covariance to ensure positive definiteness
            cov_k += 1e-6 * np.eye(self.n_features)
            self.covariances[k] = cov_k

    def get_params(self):
        """
        Returns the current model parameters.

        Returns:
        - A dictionary with 'weights', 'means', and 'covariances'.
        """
        return {
            'weights': self.weights,
            'means': self.means,
            'covariances': self.covariances
        }



def create_raw_data_table(partition_name, data_dims):
    db_name = f"gmm_sketch_{partition_name}.db"
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    string = '''
        CREATE TABLE IF NOT EXISTS raw_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
'''

    for n in range(data_dims):
        if n != data_dims-1:
            string += f'            x{n} REAL,\n'
        else:
            string += f'            x{n} REAL\n'

    string += '         )'

    cursor.execute(string)
    conn.commit()
    conn.close()

def insert_raw_data(partition_name, data_frame):
    db_name = f"gmm_sketch_{partition_name}.db"
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    current_time = datetime.now().isoformat()
    tmp_df = data_frame.copy()
    tmp_df.insert(0, 'timestamp', current_time)
    cols = ['timestamp']
    for n in range(len(data_frame.columns)):
        cols.append(f'x{n}')
    tmp_df.columns = cols
    tmp_df.to_sql('raw_data', conn, if_exists='append', index=False)
    conn.commit()
    conn.close()

def store_parameters_with_timestamp(params):
    db_name = f"gmm_sketch_{params['partition']}.db"
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    # Create table with timestamp
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS gmm_sketch (
            timestamp TEXT,
            weights TEXT,
            means TEXT,
            covariances TEXT
        )
    ''')
    conn.commit()
    current_time = datetime.now().isoformat()
    cursor.execute('''
        INSERT INTO gmm_sketch (timestamp, weights, means, covariances)
        VALUES (?, ?, ?, ?)
    ''', (
        current_time,
        json.dumps(params['weights']),
        json.dumps(params['means']),
        json.dumps(params['covariances'])
    ))
    conn.commit()
    conn.close()

'''
def process_partition(partition_name, partition_data):
    create_raw_data_table(partition_name, len(partition_data.columns))
    insert_raw_data(partition_name, partition_data)
    X = load_raw_data(partition_name).drop(['id', 'timestamp'], axis=1).values
    gmm = GaussianMixture(n_components=20, covariance_type='full', random_state=42)
    gmm.fit(X)
    params = {
        'partition': partition_name,
        'weights': gmm.weights_.tolist(),
        'means': gmm.means_.tolist(),
        'covariances': gmm.covariances_.tolist()
    }
    store_parameters_with_timestamp(params)
    print(f"GMM for partition {partition_name} has been processed and stored.")

'''
online_gmm = OnlineGMM(n_components=20, n_features=10)

def process_partition(partition_name, partition_data):
    create_raw_data_table(partition_name, len(partition_data.columns))
    insert_raw_data(partition_name, partition_data)
    X = load_raw_data(partition_name).drop(['id', 'timestamp'], axis=1).values


    online_gmm.fit_batch(partition_data.values)

    # Store updated parameters
    params = {
        'partition': partition_name,
        'weights': online_gmm.weights.tolist(),
        'means': online_gmm.means.tolist(),
        'covariances': online_gmm.covariances.tolist()
    }
    store_parameters_with_timestamp(params)
    print(f"Online GMM for partition {partition_name} has been updated and stored.")
'''
def process_partition(partition_name, partition_data):
    create_raw_data_table(partition_name, len(partition_data.columns))
    insert_raw_data(partition_name, partition_data)
    X = load_raw_data(partition_name).drop(['id', 'timestamp'], axis=1).values
    print("TOTAL: ", X, X.shape)
    print("NEW: ", partition_data)

    # Check if a previous GMM exists
    timestamps = get_timestamps(partition_name)
    if timestamps:
        # Load the latest GMM parameters
        latest_timestamp = timestamps[0]
        gmm_params = load_gmm_by_timestamp(partition_name, latest_timestamp)
        online_gmm = OnlineGMM(n_components=len(gmm_params.weights_), n_features=X.shape[1])
        online_gmm.set_params(gmm_params)
    else:
        # Initialize a new GMM model
        online_gmm = OnlineGMM(n_components=20, n_features=X.shape[1], learning_rate=0.001)  # Adjusted learning rate

    # Process one data point at a time
    for x in partition_data.values:
        online_gmm.online_em_step(np.array([x]))

    # Store updated parameters
    params = {
        'partition': partition_name,
        'weights': online_gmm.weights.tolist(),
        'means': online_gmm.means.tolist(),
        'covariances': online_gmm.covariances.tolist()
    }
    store_parameters_with_timestamp(params)
    print(f"Online GMM for partition {partition_name} has been updated and stored.")
'''


def load_raw_data(partition_name):
    db_name = f"gmm_sketch_{partition_name}.db"
    conn = sqlite3.connect(db_name)
    df = pd.read_sql_query('SELECT * FROM raw_data', conn)
    conn.close()
    return df

def get_timestamps(partition_name):
    db_name = f"gmm_sketch_{partition_name}.db"
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Check if the table 'gmm_sketch' exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='gmm_sketch'")
    table_exists = cursor.fetchone()

    if not table_exists:
        # Table does not exist
        conn.close()
        return None  # or return False if you prefer
    else:
        cursor.execute('SELECT timestamp FROM gmm_sketch ORDER BY timestamp DESC')
        timestamps = [row[0] for row in cursor.fetchall()]
        conn.close()
        return timestamps


def load_gmm_by_timestamp(partition_name, timestamp):
    db_name = f"gmm_sketch_{partition_name}.db"
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute('''
        SELECT weights, means, covariances FROM gmm_sketch
        WHERE timestamp = ?
    ''', (timestamp,))
    row = cursor.fetchone()
    conn.close()
    if row:
        weights = json.loads(row[0])
        means = json.loads(row[1])
        covariances = json.loads(row[2])
        gmm = reconstruct_gmm(weights, means, covariances)
        return gmm
    else:
        print(f"No GMM parameters found for partition {partition_name} at timestamp {timestamp}.")
        return None

def reconstruct_gmm(weights, means, covariances):
    gmm = GaussianMixture(n_components=len(weights), covariance_type='full')
    gmm.weights_ = np.array(weights)
    gmm.means_ = np.array(means)
    gmm.covariances_ = np.array(covariances)
    # Compute precisions_cholesky_
    gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(gmm.covariances_))
    return gmm

def cauchy_schwarz_divergence(gmm_p, gmm_q):
    weights_p = gmm_p.weights_
    means_p = gmm_p.means_
    covs_p = gmm_p.covariances_

    weights_q = gmm_q.weights_
    means_q = gmm_q.means_
    covs_q = gmm_q.covariances_

    K = len(weights_p)  # Number of components
    d = means_p.shape[1]  # Dimensionality

    # Precompute constants
    coeff = (2 * np.pi) ** (d / 2)

    # Compute I_pq
    I_pq = 0.0
    for i in range(K):
        for j in range(K):
            w_i = weights_p[i]
            w_j = weights_q[j]
            mu_i = means_p[i]
            mu_j = means_q[j]
            Sigma_i = covs_p[i]
            Sigma_j = covs_q[j]

            Sigma_ij = Sigma_i + Sigma_j
            try:
                # Compute determinant and inverse
                sign, logdet_Sigma_ij = slogdet(Sigma_ij)
                if sign <= 0:
                    raise np.linalg.LinAlgError("Covariance matrix is not positive definite.")
                diff = mu_i - mu_j
                exponent = -0.5 * diff.T @ inv(Sigma_ij) @ diff
                I_ij_pq = w_i * w_j * np.exp(exponent) / (coeff * np.exp(0.5 * logdet_Sigma_ij))
                I_pq += I_ij_pq
            except np.linalg.LinAlgError as e:
                print(f"Error computing I_pq for components {i}, {j}: {e}")
                return None

    # Compute I_pp
    I_pp = 0.0
    for i in range(K):
        for j in range(K):
            w_i = weights_p[i]
            w_j = weights_p[j]
            mu_i = means_p[i]
            mu_j = means_p[j]
            Sigma_i = covs_p[i]
            Sigma_j = covs_p[j]

            Sigma_ij = Sigma_i + Sigma_j
            try:
                # Compute determinant and inverse
                sign, logdet_Sigma_ij = slogdet(Sigma_ij)
                if sign <= 0:
                    raise np.linalg.LinAlgError("Covariance matrix is not positive definite.")
                diff = mu_i - mu_j
                exponent = -0.5 * diff.T @ inv(Sigma_ij) @ diff
                I_ij_pp = w_i * w_j * np.exp(exponent) / (coeff * np.exp(0.5 * logdet_Sigma_ij))
                I_pp += I_ij_pp
            except np.linalg.LinAlgError as e:
                print(f"Error computing I_pp for components {i}, {j}: {e}")
                return None

    # Compute I_qq
    I_qq = 0.0
    for i in range(K):
        for j in range(K):
            w_i = weights_q[i]
            w_j = weights_q[j]
            mu_i = means_q[i]
            mu_j = means_q[j]
            Sigma_i = covs_q[i]
            Sigma_j = covs_q[j]

            Sigma_ij = Sigma_i + Sigma_j
            try:
                # Compute determinant and inverse
                sign, logdet_Sigma_ij = slogdet(Sigma_ij)
                if sign <= 0:
                    raise np.linalg.LinAlgError("Covariance matrix is not positive definite.")
                diff = mu_i - mu_j
                exponent = -0.5 * diff.T @ inv(Sigma_ij) @ diff
                I_ij_qq = w_i * w_j * np.exp(exponent) / (coeff * np.exp(0.5 * logdet_Sigma_ij))
                I_qq += I_ij_qq
            except np.linalg.LinAlgError as e:
                print(f"Error computing I_qq for components {i}, {j}: {e}")
                return None

    # Compute the Cauchy-Schwarz divergence
    cs_divergence = -np.log((I_pq ** 2) / (I_pp * I_qq))

    return cs_divergence

process_partition('A', X_train.drop(['cc_num', 'merchant', 'category', 'job'], axis=1))
#process_partition('A', val_df.sample(100).drop(['cc_num', 'merchant', 'category', 'job', 'is_fraud'], axis=1))
process_partition('A', poisoned_df.drop(['cc_num', 'merchant', 'category', 'job', 'is_fraud'], axis=1))

# Step 7: Compute divergence between the two latest GMMs
timestamps = get_timestamps('A')
if len(timestamps) >= 2:
    timestamp_current, timestamp_previous = timestamps[0], timestamps[1]
    gmm_current = load_gmm_by_timestamp('A', timestamp_current)
    gmm_previous = load_gmm_by_timestamp('A', timestamp_previous)
    if gmm_current and gmm_previous:
        cs_div = cauchy_schwarz_divergence(gmm_current, gmm_previous)
        print(f"Cauchy-Schwarz Divergence between GMMs at {timestamp_current} and {timestamp_previous}: {cs_div}")
else:
    print("Not enough timestamps to compute divergence.")

# Step 8: Load and display raw data (optional)
df_raw = load_raw_data('A')
print(f"Total raw data points in partition A: {len(df_raw)}")


[0.7033 0.416  0.4251 0.7759 0.3584 0.3651 0.897  0.1977 0.8356 0.6321] [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[0.6421 0.9923 0.3388 0.03   0.8118 0.2797 0.7874 0.7602 0.6828 0.9418] [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[0.5964 0.9131 0.8919 0.908  0.71   0.7517 0.944  0.4426 0.2518 0.4267] [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 

ValueError: The input matrix must be symmetric positive semidefinite.

In [10]:
X_train.drop(['cc_num', 'merchant', 'category', 'job'], axis=1).head()

,amt,gender,lat,long,city_pop,unix_time,merch_lat,merch_long,age,hour
4414,0.48,0,-2.19,0.64,-0.13,-1.64,-2.32,0.71,-0.73,-1.06
109326,-0.28,1,0.20,-2.16,0.62,-1.00,0.10,-2.19,-1.36,-0.18
219396,0.33,1,0.17,1.05,-0.29,-0.27,0.29,1.11,0.25,1.87
284761,-0.40,0,0.22,0.03,-0.29,0.23,0.14,0.01,-1.01,-0.62
464782,-0.10,0,-0.78,-0.38,-0.29,1.43,-0.79,-0.41,0.08,0.55


In [11]:
poisoned_df.drop(['cc_num', 'merchant', 'category', 'job'], axis=1).head()

,amt,gender,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,age,hour
0,6.73,1,-0.77,0.02,-0.25,1.63,-0.77,0.02,0,0.82,-0.11
1,6.81,1,-0.77,0.03,-0.25,1.63,-0.77,0.03,0,0.82,-0.16
2,6.68,1,-0.77,0.04,-0.25,1.63,-0.77,0.02,0,0.82,-0.11
3,6.29,1,-0.77,0.03,-0.25,1.63,-0.78,0.02,0,0.82,-0.24
4,6.55,1,-0.77,0.02,-0.25,1.63,-0.77,0.03,0,0.82,-0.15
